In [ ]:

# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully")
print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)


Libraries imported successfully
NumPy version: 2.2.3
Pandas version: 2.2.3


In [ ]:

# Load the design datasets
partial_graft = pd.read_csv('partial_graft_designs.csv')
diversified_cdr = pd.read_csv('diversified_cdr_designs.csv')

print("=== Partial Graft Designs ===")
print(f"Shape: {partial_graft.shape}")
print(f"Columns: {list(partial_graft.columns)}")
print("\nFirst few rows:")
print(partial_graft.head())

print("\n=== Diversified CDR Designs ===")
print(f"Shape: {diversified_cdr.shape}")
print(f"Columns: {list(diversified_cdr.columns)}")
print("\nFirst few rows:")
print(diversified_cdr.head())


=== Partial Graft Designs ===
Shape: (964, 12)
Columns: ['Scaffold', 'Loop_Position', 'Loop_Range', 'Parent_Antibody', 'Parent_CDR', 'Parent_Sequence', 'Subsequence', 'Subseq_Length', 'Design_Sequence', 'Epitope_Type', 'Sequence_Score', 'Normalized_Sequence_Score']

First few rows:
  Scaffold  Loop_Position Loop_Range Parent_Antibody Parent_CDR  \
0     1TEN              2      40-48             1E5         H3   
1     1FNA              3      60-68             1E5         H3   
2     1TEN              1      15-23             1E5         H3   
3     2QMT              3      45-53             1E5         H3   
4     2QMT              2      30-38             1E5         H3   

     Parent_Sequence Subsequence  Subseq_Length  \
0  ARDYQYYYSGSYPTPHN    ARDYQYYY              8   
1  ARDYQYYYSGSYPTPHN    ARDYQYYY              8   
2  ARDYQYYYSGSYPTPHN    ARDYQYYY              8   
3  ARDYQYYYSGSYPTPHN    ARDYQYYY              8   
4  ARDYQYYYSGSYPTPHN    ARDYQYYY              8   

       

In [ ]:

# Filter for competitive epitope designs only
print("=== Filtering for Competitive Epitope Designs ===")

# Partial graft: filter by Epitope_Type
partial_competitive = partial_graft[partial_graft['Epitope_Type'] == 'competitive'].copy()
print(f"Partial graft competitive designs: {len(partial_competitive)}")

# Diversified CDR: filter by source
diversified_competitive = diversified_cdr[diversified_cdr['source'] == 'competitive'].copy()
print(f"Diversified CDR competitive designs: {len(diversified_competitive)}")

# Extract motifs from each dataset
# For partial_graft: the motif is in 'Subsequence'
# For diversified_cdr: the motif is in 'variant_cdr_sequence'

print("\n=== Extracting Motifs ===")
print(f"Partial graft motif column: 'Subsequence'")
print(f"Sample motifs: {partial_competitive['Subsequence'].head().tolist()}")

print(f"\nDiversified CDR motif column: 'variant_cdr_sequence'")
print(f"Sample motifs: {diversified_competitive['variant_cdr_sequence'].head().tolist()}")


=== Filtering for Competitive Epitope Designs ===
Partial graft competitive designs: 405
Diversified CDR competitive designs: 4683

=== Extracting Motifs ===
Partial graft motif column: 'Subsequence'
Sample motifs: ['ARDYQYYY', 'ARDYQYYY', 'ARDYQYYY', 'ARDYQYYY', 'ARDYQYYY']

Diversified CDR motif column: 'variant_cdr_sequence'
Sample motifs: ['GGAISDTYRW', 'GGAISDTYRW', 'GGAISDTYRW', 'GGNISDTYRW', 'GGNISDTYRW']


In [ ]:

# Consolidate all unique motifs from both datasets
print("=== Consolidating Unique Motifs ===")

# Extract motifs
partial_motifs = partial_competitive['Subsequence'].tolist()
diversified_motifs = diversified_competitive['variant_cdr_sequence'].tolist()

# Combine all motifs
all_motifs = partial_motifs + diversified_motifs
print(f"Total motifs (with duplicates): {len(all_motifs)}")

# Get unique motifs
unique_motifs = list(set(all_motifs))
print(f"Unique motifs: {len(unique_motifs)}")

# Create a dataframe to track motif sources
motif_data = []
for motif in unique_motifs:
    count_partial = partial_motifs.count(motif)
    count_diversified = diversified_motifs.count(motif)
    motif_data.append({
        'motif': motif,
        'length': len(motif),
        'count_partial': count_partial,
        'count_diversified': count_diversified,
        'total_count': count_partial + count_diversified
    })

motif_df = pd.DataFrame(motif_data)
print(f"\nMotif length distribution:")
print(motif_df['length'].value_counts().sort_index())
print(f"\nMotif length range: {motif_df['length'].min()} - {motif_df['length'].max()}")
print(f"\nSample of motif data:")
print(motif_df.head(10))


=== Consolidating Unique Motifs ===
Total motifs (with duplicates): 5088
Unique motifs: 1608

Motif length distribution:
length
8      26
9      21
10    431
12    809
17    321
Name: count, dtype: int64

Motif length range: 8 - 17

Sample of motif data:
               motif  length  count_partial  count_diversified  total_count
0  ARDFQYYYTGSYPTPHN      17              0                  3            3
1       RASQSIVHSNGD      12              0                  3            3
2       RSSQSIVYSNGS      12              0                  3            3
3       RSSRGIIDYLSW      12              0                  3            3
4       RSSRSIVHSNGD      12              0                  3            3
5       KSSQSIVHSHGN      12              0                  3            3
6       RSSESIVHSNGS      12              0                  3            3
7  ARDYQYWYSGTYPTPHN      17              0                  3            3
8         GFSLSSFDIS      10              0                  

In [ ]:

# Define the competitive epitope and implement Miyazawa-Jernigan scoring
COMPETITIVE_EPITOPE = "SCSRGVSKQRIIGVGEVLDR"

# Miyazawa-Jernigan contact energy matrix (in units of kT)
MJ_MATRIX = {
    'C': {'C': -24.68, 'M': -13.29, 'F': -13.61, 'I': -13.81, 'L': -14.09, 'V': -12.96, 'W': -13.60, 'Y': -10.89, 'A': -11.85, 'G': -11.01, 'T': -11.69, 'S': -10.81, 'N': -10.59, 'Q': -11.06, 'D': -11.52, 'E': -11.18, 'H': -10.00, 'R': -9.17, 'K': -9.68, 'P': -9.91},
    'M': {'C': -13.29, 'M': -10.45, 'F': -9.05, 'I': -8.90, 'L': -9.68, 'V': -8.80, 'W': -8.95, 'Y': -7.80, 'A': -7.68, 'G': -6.98, 'T': -7.91, 'S': -7.28, 'N': -6.83, 'Q': -7.28, 'D': -6.98, 'E': -7.49, 'H': -7.49, 'R': -5.80, 'K': -6.21, 'P': -6.52},
    'F': {'C': -13.61, 'M': -9.05, 'F': -9.03, 'I': -8.74, 'L': -9.03, 'V': -8.59, 'W': -8.79, 'Y': -7.26, 'A': -7.49, 'G': -7.18, 'T': -7.28, 'S': -6.82, 'N': -6.68, 'Q': -7.20, 'D': -6.87, 'E': -7.28, 'H': -7.08, 'R': -5.41, 'K': -5.80, 'P': -6.03},
    'I': {'C': -13.81, 'M': -8.90, 'F': -8.74, 'I': -8.83, 'L': -9.37, 'V': -8.73, 'W': -8.24, 'Y': -6.83, 'A': -7.28, 'G': -6.83, 'T': -7.08, 'S': -6.62, 'N': -6.21, 'Q': -6.52, 'D': -6.37, 'E': -6.83, 'H': -6.83, 'R': -4.74, 'K': -5.31, 'P': -5.62},
    'L': {'C': -14.09, 'M': -9.68, 'F': -9.03, 'I': -9.37, 'L': -9.67, 'V': -9.22, 'W': -8.49, 'Y': -7.18, 'A': -7.49, 'G': -7.18, 'T': -7.39, 'S': -6.98, 'N': -6.52, 'Q': -6.83, 'D': -6.52, 'E': -7.08, 'H': -6.83, 'R': -5.21, 'K': -5.67, 'P': -5.98},
    'V': {'C': -12.96, 'M': -8.80, 'F': -8.59, 'I': -8.73, 'L': -9.22, 'V': -8.38, 'W': -7.91, 'Y': -6.47, 'A': -6.98, 'G': -6.32, 'T': -6.83, 'S': -6.37, 'N': -5.98, 'Q': -6.42, 'D': -6.06, 'E': -6.52, 'H': -6.52, 'R': -4.69, 'K': -5.16, 'P': -5.36},
    'W': {'C': -13.60, 'M': -8.95, 'F': -8.79, 'I': -8.24, 'L': -8.49, 'V': -7.91, 'W': -8.98, 'Y': -7.28, 'A': -6.98, 'G': -7.18, 'T': -7.08, 'S': -6.47, 'N': -6.52, 'Q': -6.62, 'D': -6.57, 'E': -6.88, 'H': -6.32, 'R': -5.52, 'K': -5.21, 'P': -5.52},
    'Y': {'C': -10.89, 'M': -7.80, 'F': -7.26, 'I': -6.83, 'L': -7.18, 'V': -6.47, 'W': -7.28, 'Y': -6.17, 'A': -6.06, 'G': -6.17, 'T': -6.06, 'S': -5.57, 'N': -5.21, 'Q': -5.67, 'D': -5.31, 'E': -5.67, 'H': -5.21, 'R': -4.17, 'K': -4.53, 'P': -4.74},
    'A': {'C': -11.85, 'M': -7.68, 'F': -7.49, 'I': -7.28, 'L': -7.49, 'V': -6.98, 'W': -6.98, 'Y': -6.06, 'A': -6.32, 'G': -6.06, 'T': -6.21, 'S': -5.83, 'N': -5.52, 'Q': -5.83, 'D': -5.47, 'E': -5.98, 'H': -5.72, 'R': -4.58, 'K': -4.99, 'P': -5.10},
    'G': {'C': -11.01, 'M': -6.98, 'F': -7.18, 'I': -6.83, 'L': -7.18, 'V': -6.32, 'W': -7.18, 'Y': -6.17, 'A': -6.06, 'G': -5.67, 'T': -5.77, 'S': -5.41, 'N': -5.21, 'Q': -5.52, 'D': -5.21, 'E': -5.62, 'H': -5.52, 'R': -4.43, 'K': -4.79, 'P': -4.79},
    'T': {'C': -11.69, 'M': -7.91, 'F': -7.28, 'I': -7.08, 'L': -7.39, 'V': -6.83, 'W': -7.08, 'Y': -6.06, 'A': -6.21, 'G': -5.77, 'T': -6.42, 'S': -5.83, 'N': -5.31, 'Q': -5.77, 'D': -5.26, 'E': -5.77, 'H': -5.57, 'R': -4.48, 'K': -4.89, 'P': -5.00},
    'S': {'C': -10.81, 'M': -7.28, 'F': -6.82, 'I': -6.62, 'L': -6.98, 'V': -6.37, 'W': -6.47, 'Y': -5.57, 'A': -5.83, 'G': -5.41, 'T': -5.83, 'S': -5.52, 'N': -5.16, 'Q': -5.47, 'D': -5.05, 'E': -5.52, 'H': -5.21, 'R': -4.33, 'K': -4.69, 'P': -4.74},
    'N': {'C': -10.59, 'M': -6.83, 'F': -6.68, 'I': -6.21, 'L': -6.52, 'V': -5.98, 'W': -6.52, 'Y': -5.21, 'A': -5.52, 'G': -5.21, 'T': -5.31, 'S': -5.16, 'N': -4.89, 'Q': -5.26, 'D': -4.89, 'E': -5.26, 'H': -5.05, 'R': -4.07, 'K': -4.38, 'P': -4.43},
    'Q': {'C': -11.06, 'M': -7.28, 'F': -7.20, 'I': -6.52, 'L': -6.83, 'V': -6.42, 'W': -6.62, 'Y': -5.67, 'A': -5.83, 'G': -5.52, 'T': -5.77, 'S': -5.47, 'N': -5.26, 'Q': -5.57, 'D': -5.16, 'E': -5.62, 'H': -5.31, 'R': -4.28, 'K': -4.64, 'P': -4.74},
    'D': {'C': -11.52, 'M': -6.98, 'F': -6.87, 'I': -6.37, 'L': -6.52, 'V': -6.06, 'W': -6.57, 'Y': -5.31, 'A': -5.47, 'G': -5.21, 'T': -5.26, 'S': -5.05, 'N': -4.89, 'Q': -5.16, 'D': -4.84, 'E': -5.21, 'H': -5.00, 'R': -3.98, 'K': -4.33, 'P': -4.43},
    'E': {'C': -11.18, 'M': -7.49, 'F': -7.28, 'I': -6.83, 'L': -7.08, 'V': -6.52, 'W': -6.88, 'Y': -5.67, 'A': -5.98, 'G': -5.62, 'T': -5.77, 'S': -5.52, 'N': -5.26, 'Q': -5.62, 'D': -5.21, 'E': -5.57, 'H': -5.31, 'R': -4.28, 'K': -4.64, 'P': -4.69},
    'H': {'C': -10.00, 'M': -7.49, 'F': -7.08, 'I': -6.83, 'L': -6.83, 'V': -6.52, 'W': -6.32, 'Y': -5.21, 'A': -5.72, 'G': -5.52, 'T': -5.57, 'S': -5.21, 'N': -5.05, 'Q': -5.31, 'D': -5.00, 'E': -5.31, 'H': -5.10, 'R': -4.12, 'K': -4.43, 'P': -4.48},
    'R': {'C': -9.17, 'M': -5.80, 'F': -5.41, 'I': -4.74, 'L': -5.21, 'V': -4.69, 'W': -5.52, 'Y': -4.17, 'A': -4.58, 'G': -4.43, 'T': -4.48, 'S': -4.33, 'N': -4.07, 'Q': -4.28, 'D': -3.98, 'E': -4.28, 'H': -4.12, 'R': -3.63, 'K': -3.93, 'P': -3.98},
    'K': {'C': -9.68, 'M': -6.21, 'F': -5.80, 'I': -5.31, 'L': -5.67, 'V': -5.16, 'W': -5.21, 'Y': -4.53, 'A': -4.99, 'G': -4.79, 'T': -4.89, 'S': -4.69, 'N': -4.38, 'Q': -4.64, 'D': -4.33, 'E': -4.64, 'H': -4.43, 'R': -3.93, 'K': -4.33, 'P': -4.33},
    'P': {'C': -9.91, 'M': -6.52, 'F': -6.03, 'I': -5.62, 'L': -5.98, 'V': -5.36, 'W': -5.52, 'Y': -4.74, 'A': -5.10, 'G': -4.79, 'T': -5.00, 'S': -4.74, 'N': -4.43, 'Q': -4.74, 'D': -4.43, 'E': -4.69, 'H': -4.48, 'R': -3.98, 'K': -4.33, 'P': -4.38}
}

def calculate_mj_score(motif, epitope):
    """Calculate Miyazawa-Jernigan interaction score between motif and epitope"""
    score = 0.0
    interactions = 0
    for m_res in motif:
        for e_res in epitope:
            if m_res in MJ_MATRIX and e_res in MJ_MATRIX[m_res]:
                score += MJ_MATRIX[m_res][e_res]
                interactions += 1
    return score, interactions

print("Competitive epitope:", COMPETITIVE_EPITOPE)
print(f"Epitope length: {len(COMPETITIVE_EPITOPE)}")
print("\nMJ scoring function defined")


Competitive epitope: SCSRGVSKQRIIGVGEVLDR
Epitope length: 20

MJ scoring function defined


In [ ]:

# Calculate MJ scores for all unique motifs against competitive epitope
print("=== Calculating MJ Scores for All Motifs ===")

scores_list = []
for motif in motif_df['motif']:
    score, interactions = calculate_mj_score(motif, COMPETITIVE_EPITOPE)
    scores_list.append({
        'motif': motif,
        'raw_mj_score': score,
        'interactions': interactions
    })

scores_df = pd.DataFrame(scores_list)

# Merge with motif_df
motif_df = motif_df.merge(scores_df, on='motif')

print(f"Scored {len(motif_df)} unique motifs")
print(f"\nRaw MJ score statistics:")
print(motif_df['raw_mj_score'].describe())
print(f"\nSample of scored motifs:")
print(motif_df[['motif', 'length', 'raw_mj_score', 'interactions']].head(10))


=== Calculating MJ Scores for All Motifs ===
Scored 1608 unique motifs

Raw MJ score statistics:
count    1608.000000
mean    -1495.417040
std       262.867196
min     -2013.520000
25%     -1527.270000
50%     -1431.210000
75%     -1285.500000
max      -907.930000
Name: raw_mj_score, dtype: float64

Sample of scored motifs:
               motif  length  raw_mj_score  interactions
0  ARDFQYYYTGSYPTPHN      17      -1989.31           340
1       RASQSIVHSNGD      12      -1427.67           240
2       RSSQSIVYSNGS      12      -1428.63           240
3       RSSRGIIDYLSW      12      -1475.56           240
4       RSSRSIVHSNGD      12      -1392.38           240
5       KSSQSIVHSHGN      12      -1429.28           240
6       RSSESIVHSNGS      12      -1425.02           240
7  ARDYQYWYSGTYPTPHN      17      -1983.13           340
8         GFSLSSFDIS      10      -1299.84           200
9       QASQGIIDWLSW      12      -1561.31           240


In [ ]:

# Implement Residual Score methodology to correct for length bias
# Fit a quadratic polynomial: raw_score = a * length^2 + b * length + c

print("=== Calculating Residual Scores (Length-Independent) ===")

# Fit polynomial regression
lengths = motif_df['length'].values
raw_scores = motif_df['raw_mj_score'].values

# Fit 2nd degree polynomial
coeffs = np.polyfit(lengths, raw_scores, 2)
print(f"Polynomial coefficients: a={coeffs[0]:.4f}, b={coeffs[1]:.4f}, c={coeffs[2]:.4f}")

# Calculate predicted scores
predicted_scores = np.polyval(coeffs, lengths)

# Calculate residual scores
residual_scores = raw_scores - predicted_scores

# Add to dataframe
motif_df['predicted_mj_score'] = predicted_scores
motif_df['residual_score'] = residual_scores

print(f"\nResidual score statistics:")
print(motif_df['residual_score'].describe())

# Visualize the fit
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Raw score vs length with fit
ax = axes[0]
ax.scatter(lengths, raw_scores, alpha=0.5, s=20)
length_range = np.linspace(lengths.min(), lengths.max(), 100)
predicted_range = np.polyval(coeffs, length_range)
ax.plot(length_range, predicted_range, 'r-', linewidth=2, label='Quadratic fit')
ax.set_xlabel('Motif Length (residues)', fontsize=11)
ax.set_ylabel('Raw MJ Score', fontsize=11)
ax.set_title('Raw MJ Score vs Length\n(showing length bias)', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# Plot 2: Residual score vs length (should be flat)
ax = axes[1]
ax.scatter(lengths, residual_scores, alpha=0.5, s=20, color='green')
ax.axhline(y=0, color='r', linestyle='--', linewidth=2)
ax.set_xlabel('Motif Length (residues)', fontsize=11)
ax.set_ylabel('Residual Score', fontsize=11)
ax.set_title('Residual Score vs Length\n(length-independent)', fontsize=12, fontweight='bold')
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('length_bias_correction.png', dpi=150, bbox_inches='tight')
plt.close()

print("\nLength bias correction plot saved")
print(f"\nTop 10 motifs by residual score:")
print(motif_df.nlargest(10, 'residual_score')[['motif', 'length', 'raw_mj_score', 'residual_score']])


=== Calculating Residual Scores (Length-Independent) ===
Polynomial coefficients: a=1.5925, b=-146.3018, c=59.3411

Residual score statistics:
count    1.608000e+03
mean    -8.571408e-12
std      4.102721e+01
min     -1.005341e+02
25%     -3.500630e+01
50%     -9.623913e-01
75%      3.577586e+01
max      1.038000e+02
Name: residual_score, dtype: float64



Length bias correction plot saved

Top 10 motifs by residual score:
          motif  length  raw_mj_score  residual_score
591   SGSYPTPHN       9      -1024.58      103.800020
674    GSYPTPHN       8       -907.93      101.221275
630    SGSYPTPH       8       -914.00       95.151275
1270  YSGSYPTPH       9      -1034.05       94.330020
419   RDYQYYYSG       9      -1035.11       93.270020
1453   RDYQYYYS       8       -916.63       92.521275
1414   YSGSYPTP       8       -919.40       89.751275
397   YYSGSYPTP       9      -1039.45       88.930020
1021  ARDYQYYYS       9      -1043.15       85.230020
1349   ARDYQYYY       8       -926.50       82.651275


In [ ]:

# Create High-Scoring and Low-Scoring groups based on residual scores
print("=== Creating High-Scoring and Low-Scoring Groups ===")

# Calculate percentiles
residual_10th_percentile = motif_df['residual_score'].quantile(0.90)  # Top 10%
residual_50th_percentile = motif_df['residual_score'].quantile(0.50)  # Bottom 50%

print(f"Top 10% threshold (90th percentile): {residual_10th_percentile:.2f}")
print(f"Bottom 50% threshold (50th percentile): {residual_50th_percentile:.2f}")

# Create groups
high_scoring = motif_df[motif_df['residual_score'] >= residual_10th_percentile].copy()
low_scoring = motif_df[motif_df['residual_score'] <= residual_50th_percentile].copy()

print(f"\nHigh-scoring group (top 10%): {len(high_scoring)} motifs")
print(f"Low-scoring group (bottom 50%): {len(low_scoring)} motifs")

print(f"\nHigh-scoring group residual score range: {high_scoring['residual_score'].min():.2f} to {high_scoring['residual_score'].max():.2f}")
print(f"Low-scoring group residual score range: {low_scoring['residual_score'].min():.2f} to {low_scoring['residual_score'].max():.2f}")

print(f"\nLength distribution in High-scoring group:")
print(high_scoring['length'].value_counts().sort_index())
print(f"\nLength distribution in Low-scoring group:")
print(low_scoring['length'].value_counts().sort_index())


=== Creating High-Scoring and Low-Scoring Groups ===
Top 10% threshold (90th percentile): 49.80
Bottom 50% threshold (50th percentile): -0.96

High-scoring group (top 10%): 162 motifs
Low-scoring group (bottom 50%): 804 motifs

High-scoring group residual score range: 49.80 to 103.80
Low-scoring group residual score range: -100.53 to -0.96

Length distribution in High-scoring group:
length
8      11
9      11
10     15
12    125
Name: count, dtype: int64

Length distribution in Low-scoring group:
length
8       5
9       5
10    256
12    361
17    177
Name: count, dtype: int64


In [ ]:

# Calculate physicochemical properties for all motifs in both groups
print("=== Calculating Physicochemical Properties ===")

def calculate_physicochemical_properties(sequence):
    """Calculate various physicochemical properties of a peptide sequence"""
    try:
        # Use Biopython's ProteinAnalysis
        analyzed_seq = ProteinAnalysis(sequence)
        
        # Calculate properties
        properties = {
            'charge_at_pH7': analyzed_seq.charge_at_pH(7.0),
            'isoelectric_point': analyzed_seq.isoelectric_point(),
            'aromaticity': analyzed_seq.aromaticity(),
            'hydrophobicity': analyzed_seq.gravy(),  # GRAVY (Grand Average of Hydropathy)
        }
        
        # Calculate fraction of positively charged residues (K, R, H)
        positive_count = sum(1 for aa in sequence if aa in 'KRH')
        properties['frac_positive'] = positive_count / len(sequence)
        
        # Calculate fraction of negatively charged residues (D, E)
        negative_count = sum(1 for aa in sequence if aa in 'DE')
        properties['frac_negative'] = negative_count / len(sequence)
        
        # Calculate fraction of aromatic residues (W, Y, F)
        aromatic_count = sum(1 for aa in sequence if aa in 'WYF')
        properties['frac_aromatic'] = aromatic_count / len(sequence)
        
        return properties
    except Exception as e:
        print(f"Error processing sequence {sequence}: {e}")
        return None

# Calculate properties for high-scoring group
print("\nCalculating properties for High-scoring group...")
high_properties = []
for motif in high_scoring['motif']:
    props = calculate_physicochemical_properties(motif)
    if props:
        props['motif'] = motif
        props['group'] = 'high'
        high_properties.append(props)

# Calculate properties for low-scoring group
print("Calculating properties for Low-scoring group...")
low_properties = []
for motif in low_scoring['motif']:
    props = calculate_physicochemical_properties(motif)
    if props:
        props['motif'] = motif
        props['group'] = 'low'
        low_properties.append(props)

# Combine into dataframes
high_props_df = pd.DataFrame(high_properties)
low_props_df = pd.DataFrame(low_properties)
all_props_df = pd.concat([high_props_df, low_props_df], ignore_index=True)

print(f"\nHigh-scoring properties calculated: {len(high_props_df)}")
print(f"Low-scoring properties calculated: {len(low_props_df)}")

print("\n=== High-Scoring Group Property Summary ===")
print(high_props_df[['charge_at_pH7', 'hydrophobicity', 'frac_aromatic', 'frac_positive', 'frac_negative', 'isoelectric_point']].describe())

print("\n=== Low-Scoring Group Property Summary ===")
print(low_props_df[['charge_at_pH7', 'hydrophobicity', 'frac_aromatic', 'frac_positive', 'frac_negative', 'isoelectric_point']].describe())


=== Calculating Physicochemical Properties ===

Calculating properties for High-scoring group...
Calculating properties for Low-scoring group...

High-scoring properties calculated: 162
Low-scoring properties calculated: 804

=== High-Scoring Group Property Summary ===
       charge_at_pH7  hydrophobicity  frac_aromatic  frac_positive  \
count     162.000000      162.000000     162.000000     162.000000   
mean        0.806378       -1.200557       0.068141       0.173885   
std         0.858034        0.221221       0.138714       0.078762   
min        -1.242774       -2.187500       0.000000       0.000000   
25%        -0.151613       -1.316667       0.000000       0.111111   
50%         0.847266       -1.166667       0.000000       0.166667   
75%         1.846267       -1.091667       0.083333       0.250000   
max         1.934430       -0.633333       0.555556       0.333333   

       frac_negative  isoelectric_point  
count     162.000000         162.000000  
mean        0.0

In [ ]:

# Perform statistical comparison using Mann-Whitney U tests
print("=== Statistical Comparison of High-Scoring vs Low-Scoring Groups ===")
print("Using Mann-Whitney U test (non-parametric) for all properties\n")

properties = ['charge_at_pH7', 'hydrophobicity', 'frac_aromatic', 'frac_positive', 'frac_negative', 'isoelectric_point']

comparison_results = []
for prop in properties:
    high_vals = high_props_df[prop].values
    low_vals = low_props_df[prop].values
    
    # Perform Mann-Whitney U test
    statistic, pvalue = stats.mannwhitneyu(high_vals, low_vals, alternative='two-sided')
    
    # Calculate medians and effect size (Cliff's Delta approximation)
    median_high = np.median(high_vals)
    median_low = np.median(low_vals)
    mean_high = np.mean(high_vals)
    mean_low = np.mean(low_vals)
    
    # Calculate Cohen's d effect size
    pooled_std = np.sqrt((np.std(high_vals, ddof=1)**2 + np.std(low_vals, ddof=1)**2) / 2)
    cohens_d = (mean_high - mean_low) / pooled_std if pooled_std > 0 else 0
    
    comparison_results.append({
        'Property': prop,
        'High_Mean': mean_high,
        'Low_Mean': mean_low,
        'High_Median': median_high,
        'Low_Median': median_low,
        'Difference_Mean': mean_high - mean_low,
        'Difference_Median': median_high - median_low,
        'Mann_Whitney_U': statistic,
        'p_value': pvalue,
        'Cohens_d': cohens_d,
        'Significant': 'Yes' if pvalue < 0.05 else 'No'
    })

results_df = pd.DataFrame(comparison_results)
print(results_df.to_string(index=False))

print("\n=== Interpretation ===")
for _, row in results_df.iterrows():
    if row['Significant'] == 'Yes':
        direction = "higher" if row['Difference_Mean'] > 0 else "lower"
        print(f"{row['Property']}: High-scoring motifs have significantly {direction} values (p={row['p_value']:.2e}, Cohen's d={row['Cohens_d']:.2f})")


=== Statistical Comparison of High-Scoring vs Low-Scoring Groups ===
Using Mann-Whitney U test (non-parametric) for all properties

         Property  High_Mean  Low_Mean  High_Median  Low_Median  Difference_Mean  Difference_Median  Mann_Whitney_U      p_value  Cohens_d Significant
    charge_at_pH7   0.806378 -0.484183     0.847266   -0.239787         1.290561           1.087052        109074.0 3.005679e-42  1.700130         Yes
   hydrophobicity  -1.200557 -0.331981    -1.166667   -0.116667        -0.868576          -1.050000         26166.0 2.616991e-33 -1.666387         Yes
    frac_aromatic   0.068141  0.198677     0.000000    0.200000        -0.130535          -0.200000         19938.0 4.043234e-47 -1.238576         Yes
    frac_positive   0.173885  0.073253     0.166667    0.083333         0.100633           0.083333        108871.5 1.106986e-43  1.456203         Yes
    frac_negative   0.025789  0.081253     0.000000    0.083333        -0.055465          -0.083333         26340

In [ ]:

# Create comprehensive visualizations comparing the two groups
print("=== Creating Comparative Visualizations ===")

# Create figure with distribution plots for each property
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

properties = ['charge_at_pH7', 'hydrophobicity', 'frac_aromatic', 'frac_positive', 'frac_negative', 'isoelectric_point']
property_labels = {
    'charge_at_pH7': 'Net Charge (pH 7)',
    'hydrophobicity': 'Hydrophobicity (GRAVY)',
    'frac_aromatic': 'Aromatic Fraction',
    'frac_positive': 'Positive Charge Fraction',
    'frac_negative': 'Negative Charge Fraction',
    'isoelectric_point': 'Isoelectric Point (pI)'
}

for idx, prop in enumerate(properties):
    ax = axes[idx]
    
    # Get data
    high_vals = high_props_df[prop].values
    low_vals = low_props_df[prop].values
    
    # Create histograms
    ax.hist(low_vals, bins=30, alpha=0.6, color='#e74c3c', label='Low-Scoring (bottom 50%)', density=True)
    ax.hist(high_vals, bins=30, alpha=0.6, color='#3498db', label='High-Scoring (top 10%)', density=True)
    
    # Add median lines
    ax.axvline(np.median(low_vals), color='#c0392b', linestyle='--', linewidth=2, label=f'Low Median: {np.median(low_vals):.2f}')
    ax.axvline(np.median(high_vals), color='#2980b9', linestyle='--', linewidth=2, label=f'High Median: {np.median(high_vals):.2f}')
    
    ax.set_xlabel(property_labels[prop], fontsize=10, fontweight='bold')
    ax.set_ylabel('Density', fontsize=10)
    ax.legend(fontsize=7, loc='best')
    ax.grid(alpha=0.3)
    
    # Add p-value annotation
    result = results_df[results_df['Property'] == prop].iloc[0]
    pval_text = f"p = {result['p_value']:.1e}\nCohen's d = {result['Cohens_d']:.2f}"
    ax.text(0.02, 0.98, pval_text, transform=ax.transAxes, 
            verticalalignment='top', fontsize=8, 
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.savefig('physicochemical_comparison_distributions.png', dpi=150, bbox_inches='tight')
plt.close()

print("Distribution comparison plot saved")


=== Creating Comparative Visualizations ===


Distribution comparison plot saved


In [ ]:

# Examine some representative high-scoring and low-scoring motifs to understand patterns
print("=== Representative Motifs from Each Group ===")

# Merge properties back with motif_df to get residual scores
high_scoring_with_props = high_scoring.merge(high_props_df, on='motif')
low_scoring_with_props = low_scoring.merge(low_props_df, on='motif')

print("\n=== Top 20 High-Scoring Competitive Motifs ===")
print("(Sorted by residual score)")
top_high = high_scoring_with_props.nlargest(20, 'residual_score')[
    ['motif', 'length', 'residual_score', 'charge_at_pH7', 'hydrophobicity', 
     'frac_aromatic', 'frac_positive', 'frac_negative', 'isoelectric_point']
]
print(top_high.to_string(index=False))

print("\n=== Bottom 20 Low-Scoring Competitive Motifs ===")
print("(Sorted by residual score)")
bottom_low = low_scoring_with_props.nsmallest(20, 'residual_score')[
    ['motif', 'length', 'residual_score', 'charge_at_pH7', 'hydrophobicity', 
     'frac_aromatic', 'frac_positive', 'frac_negative', 'isoelectric_point']
]
print(bottom_low.to_string(index=False))


=== Representative Motifs from Each Group ===

=== Top 20 High-Scoring Competitive Motifs ===
(Sorted by residual score)
       motif  length  residual_score  charge_at_pH7  hydrophobicity  frac_aromatic  frac_positive  frac_negative  isoelectric_point
   SGSYPTPHN       9      103.800020      -0.453678       -1.544444       0.111111       0.111111       0.000000           6.456080
    GSYPTPHN       8      101.221275      -0.153723       -1.637500       0.125000       0.125000       0.000000           6.740161
    SGSYPTPH       8       95.151275      -0.453678       -1.300000       0.125000       0.125000       0.000000           6.456080
   YSGSYPTPH       9       94.330020      -0.154722       -1.300000       0.222222       0.111111       0.000000           6.739252
   RDYQYYYSG       9       93.270020      -0.242784       -1.988889       0.444444       0.111111       0.111111           5.833522
    RDYQYYYS       8       92.521275      -0.242784       -2.187500       0.500000     

In [ ]:

# Now let's compare with allosteric binders to define contrasting design grammars
# We need to load and analyze allosteric motifs from the diversified_cdr dataset

print("=== Analyzing Allosteric Motifs for Comparison ===")

# Filter for allosteric designs
diversified_allosteric = diversified_cdr[diversified_cdr['source'] == 'allosteric'].copy()
print(f"Diversified CDR allosteric designs: {len(diversified_allosteric)}")

# Get unique allosteric motifs
allosteric_motifs = diversified_allosteric['variant_cdr_sequence'].unique()
print(f"Unique allosteric motifs: {len(allosteric_motifs)}")

# Calculate properties for allosteric motifs
print("\nCalculating properties for allosteric motifs...")
allosteric_properties = []
for motif in allosteric_motifs:
    props = calculate_physicochemical_properties(motif)
    if props:
        props['motif'] = motif
        props['group'] = 'allosteric'
        allosteric_properties.append(props)

allosteric_props_df = pd.DataFrame(allosteric_properties)
print(f"Allosteric properties calculated: {len(allosteric_props_df)}")

print("\n=== Allosteric Motif Property Summary ===")
print(allosteric_props_df[['charge_at_pH7', 'hydrophobicity', 'frac_aromatic', 'frac_positive', 'frac_negative', 'isoelectric_point']].describe())


=== Analyzing Allosteric Motifs for Comparison ===
Diversified CDR allosteric designs: 7560
Unique allosteric motifs: 2520

Calculating properties for allosteric motifs...


Allosteric properties calculated: 2520

=== Allosteric Motif Property Summary ===
       charge_at_pH7  hydrophobicity  frac_aromatic  frac_positive  \
count    2520.000000     2520.000000    2520.000000    2520.000000   
mean       -0.629317       -0.534017       0.165186       0.115874   
std         1.258946        0.664843       0.058096       0.064119   
min        -3.502114       -1.958333       0.000000       0.000000   
25%        -2.074187       -1.020000       0.100000       0.083333   
50%        -0.538042       -0.580000       0.166667       0.100000   
75%         0.491256       -0.230000       0.200000       0.166667   
max         2.747194        1.718182       0.250000       0.333333   

       frac_negative  isoelectric_point  
count    2520.000000        2520.000000  
mean        0.105899           6.169780  
std         0.069834           1.879505  
min         0.000000           4.050028  
25%         0.083333           4.353321  
50%         0.100000           5.69

In [ ]:

# Statistical comparison: High-scoring competitive vs Allosteric motifs
print("=== Statistical Comparison: High-Scoring Competitive vs Allosteric ===")
print("Using Mann-Whitney U test (non-parametric) for all properties\n")

properties = ['charge_at_pH7', 'hydrophobicity', 'frac_aromatic', 'frac_positive', 'frac_negative', 'isoelectric_point']

competitive_vs_allosteric_results = []
for prop in properties:
    high_competitive_vals = high_props_df[prop].values
    allosteric_vals = allosteric_props_df[prop].values
    
    # Perform Mann-Whitney U test
    statistic, pvalue = stats.mannwhitneyu(high_competitive_vals, allosteric_vals, alternative='two-sided')
    
    # Calculate medians and means
    median_competitive = np.median(high_competitive_vals)
    median_allosteric = np.median(allosteric_vals)
    mean_competitive = np.mean(high_competitive_vals)
    mean_allosteric = np.mean(allosteric_vals)
    
    # Calculate Cohen's d effect size
    pooled_std = np.sqrt((np.std(high_competitive_vals, ddof=1)**2 + np.std(allosteric_vals, ddof=1)**2) / 2)
    cohens_d = (mean_competitive - mean_allosteric) / pooled_std if pooled_std > 0 else 0
    
    competitive_vs_allosteric_results.append({
        'Property': prop,
        'Competitive_Mean': mean_competitive,
        'Allosteric_Mean': mean_allosteric,
        'Competitive_Median': median_competitive,
        'Allosteric_Median': median_allosteric,
        'Difference_Mean': mean_competitive - mean_allosteric,
        'Difference_Median': median_competitive - median_allosteric,
        'Mann_Whitney_U': statistic,
        'p_value': pvalue,
        'Cohens_d': cohens_d,
        'Significant': 'Yes' if pvalue < 0.05 else 'No'
    })

comp_vs_allo_df = pd.DataFrame(competitive_vs_allosteric_results)
print(comp_vs_allo_df.to_string(index=False))

print("\n=== Interpretation ===")
for _, row in comp_vs_allo_df.iterrows():
    if row['Significant'] == 'Yes':
        direction = "higher" if row['Difference_Mean'] > 0 else "lower"
        print(f"{row['Property']}: High-scoring competitive motifs have significantly {direction} values than allosteric motifs (p={row['p_value']:.2e}, Cohen's d={row['Cohens_d']:.2f})")


=== Statistical Comparison: High-Scoring Competitive vs Allosteric ===
Using Mann-Whitney U test (non-parametric) for all properties

         Property  Competitive_Mean  Allosteric_Mean  Competitive_Median  Allosteric_Median  Difference_Mean  Difference_Median  Mann_Whitney_U      p_value  Cohens_d Significant
    charge_at_pH7          0.806378        -0.629317            0.847266          -0.538042         1.435695           1.385308        346172.5 5.065576e-50  1.332674         Yes
   hydrophobicity         -1.200557        -0.534017           -1.166667          -0.580000        -0.666540          -0.586667         68960.5 1.941388e-45 -1.345304         Yes
    frac_aromatic          0.068141         0.165186            0.000000           0.166667        -0.097044          -0.166667         74128.0 1.655569e-43 -0.912581         Yes
    frac_positive          0.173885         0.115874            0.166667           0.100000         0.058011           0.066667        294658.0 4.0703

In [ ]:

# Create a comprehensive summary table comparing all three groups
print("=== Comprehensive Three-Way Comparison Summary ===")

# Create summary table
summary_data = []

for prop in properties:
    summary_data.append({
        'Property': property_labels[prop],
        'High_Competitive_Mean': high_props_df[prop].mean(),
        'Low_Competitive_Mean': low_props_df[prop].mean(),
        'Allosteric_Mean': allosteric_props_df[prop].mean(),
        'High_vs_Low_p': results_df[results_df['Property'] == prop]['p_value'].values[0],
        'High_vs_Allo_p': comp_vs_allo_df[comp_vs_allo_df['Property'] == prop]['p_value'].values[0],
        'High_vs_Low_d': results_df[results_df['Property'] == prop]['Cohens_d'].values[0],
        'High_vs_Allo_d': comp_vs_allo_df[comp_vs_allo_df['Property'] == prop]['Cohens_d'].values[0]
    })

summary_table = pd.DataFrame(summary_data)
print("\nMean Values Comparison:")
print(summary_table[['Property', 'High_Competitive_Mean', 'Low_Competitive_Mean', 'Allosteric_Mean']].to_string(index=False))

print("\n\nStatistical Significance and Effect Sizes:")
print(summary_table[['Property', 'High_vs_Low_p', 'High_vs_Low_d', 'High_vs_Allo_p', 'High_vs_Allo_d']].to_string(index=False))


=== Comprehensive Three-Way Comparison Summary ===

Mean Values Comparison:
                Property  High_Competitive_Mean  Low_Competitive_Mean  Allosteric_Mean
       Net Charge (pH 7)               0.806378             -0.484183        -0.629317
  Hydrophobicity (GRAVY)              -1.200557             -0.331981        -0.534017
       Aromatic Fraction               0.068141              0.198677         0.165186
Positive Charge Fraction               0.173885              0.073253         0.115874
Negative Charge Fraction               0.025789              0.081253         0.105899
  Isoelectric Point (pI)               9.029017              5.712317         6.169780


Statistical Significance and Effect Sizes:
                Property  High_vs_Low_p  High_vs_Low_d  High_vs_Allo_p  High_vs_Allo_d
       Net Charge (pH 7)   3.005679e-42       1.700130    5.065576e-50        1.332674
  Hydrophobicity (GRAVY)   2.616991e-33      -1.666387    1.941388e-45       -1.345304
       Ar

In [ ]:

# Create the final summary figure showing key physicochemical differences
print("=== Creating Final Summary Figure ===")

# Create a single-column figure with 2 subplots stacked vertically
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12))

# Panel A: Bar chart comparing mean values across three groups
properties_short = ['Net Charge', 'Hydrophobicity', 'Aromatic\nFraction', 
                   'Positive\nCharge', 'Negative\nCharge', 'pI']

high_comp_means = [
    high_props_df['charge_at_pH7'].mean(),
    high_props_df['hydrophobicity'].mean(),
    high_props_df['frac_aromatic'].mean(),
    high_props_df['frac_positive'].mean(),
    high_props_df['frac_negative'].mean(),
    high_props_df['isoelectric_point'].mean()
]

low_comp_means = [
    low_props_df['charge_at_pH7'].mean(),
    low_props_df['hydrophobicity'].mean(),
    low_props_df['frac_aromatic'].mean(),
    low_props_df['frac_positive'].mean(),
    low_props_df['frac_negative'].mean(),
    low_props_df['isoelectric_point'].mean()
]

allosteric_means = [
    allosteric_props_df['charge_at_pH7'].mean(),
    allosteric_props_df['hydrophobicity'].mean(),
    allosteric_props_df['frac_aromatic'].mean(),
    allosteric_props_df['frac_positive'].mean(),
    allosteric_props_df['frac_negative'].mean(),
    allosteric_props_df['isoelectric_point'].mean()
]

# Normalize each property to [-1, 1] range for visualization
normalized_high = []
normalized_low = []
normalized_allo = []

for i in range(len(high_comp_means)):
    all_vals = [high_comp_means[i], low_comp_means[i], allosteric_means[i]]
    min_val = min(all_vals)
    max_val = max(all_vals)
    range_val = max_val - min_val if max_val != min_val else 1
    
    normalized_high.append(2 * (high_comp_means[i] - min_val) / range_val - 1)
    normalized_low.append(2 * (low_comp_means[i] - min_val) / range_val - 1)
    normalized_allo.append(2 * (allosteric_means[i] - min_val) / range_val - 1)

x = np.arange(len(properties_short))
width = 0.25

bars1 = ax1.bar(x - width, normalized_high, width, label='High-Scoring Competitive', 
                color='#2ecc71', edgecolor='black', linewidth=1.5)
bars2 = ax1.bar(x, normalized_low, width, label='Low-Scoring Competitive', 
                color='#e74c3c', edgecolor='black', linewidth=1.5)
bars3 = ax1.bar(x + width, normalized_allo, width, label='Allosteric', 
                color='#f39c12', edgecolor='black', linewidth=1.5)

ax1.set_ylabel('Normalized Mean Value', fontsize=12, fontweight='bold')
ax1.set_xlabel('Physicochemical Property', fontsize=12, fontweight='bold')
ax1.set_title('A. Physicochemical Profile Comparison:\nHigh-Scoring Competitive vs Low-Scoring Competitive vs Allosteric', 
              fontsize=13, fontweight='bold', pad=15)
ax1.set_xticks(x)
ax1.set_xticklabels(properties_short, fontsize=10)
ax1.legend(fontsize=10, loc='upper left')
ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax1.grid(axis='y', alpha=0.3)
ax1.set_ylim(-1.2, 1.2)

# Panel B: Effect size comparison (Cohen's d)
effect_sizes_high_vs_low = [
    results_df[results_df['Property'] == 'charge_at_pH7']['Cohens_d'].values[0],
    results_df[results_df['Property'] == 'hydrophobicity']['Cohens_d'].values[0],
    results_df[results_df['Property'] == 'frac_aromatic']['Cohens_d'].values[0],
    results_df[results_df['Property'] == 'frac_positive']['Cohens_d'].values[0],
    results_df[results_df['Property'] == 'frac_negative']['Cohens_d'].values[0],
    results_df[results_df['Property'] == 'isoelectric_point']['Cohens_d'].values[0]
]

effect_sizes_high_vs_allo = [
    comp_vs_allo_df[comp_vs_allo_df['Property'] == 'charge_at_pH7']['Cohens_d'].values[0],
    comp_vs_allo_df[comp_vs_allo_df['Property'] == 'hydrophobicity']['Cohens_d'].values[0],
    comp_vs_allo_df[comp_vs_allo_df['Property'] == 'frac_aromatic']['Cohens_d'].values[0],
    comp_vs_allo_df[comp_vs_allo_df['Property'] == 'frac_positive']['Cohens_d'].values[0],
    comp_vs_allo_df[comp_vs_allo_df['Property'] == 'frac_negative']['Cohens_d'].values[0],
    comp_vs_allo_df[comp_vs_allo_df['Property'] == 'isoelectric_point']['Cohens_d'].values[0]
]

bars4 = ax2.bar(x - width/2, effect_sizes_high_vs_low, width, 
                label='High vs Low Competitive', color='#3498db', 
                edgecolor='black', linewidth=1.5)
bars5 = ax2.bar(x + width/2, effect_sizes_high_vs_allo, width, 
                label='High Competitive vs Allosteric', color='#9b59b6', 
                edgecolor='black', linewidth=1.5)

ax2.set_ylabel("Cohen's d Effect Size", fontsize=12, fontweight='bold')
ax2.set_xlabel('Physicochemical Property', fontsize=12, fontweight='bold')
ax2.set_title("B. Effect Sizes: High-Scoring Competitive Motifs vs Other Groups", 
              fontsize=13, fontweight='bold', pad=15)
ax2.set_xticks(x)
ax2.set_xticklabels(properties_short, fontsize=10)
ax2.legend(fontsize=10, loc='upper right')
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax2.axhline(y=0.8, color='gray', linestyle='--', linewidth=1, alpha=0.5, label='Large effect')
ax2.axhline(y=-0.8, color='gray', linestyle='--', linewidth=1, alpha=0.5)
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('competitive_binder_physicochemical_signature.png', dpi=300, bbox_inches='tight')
print("Final summary figure saved: competitive_binder_physicochemical_signature.png")
plt.close()

print("\n=== Analysis Complete ===")


=== Creating Final Summary Figure ===


Final summary figure saved: competitive_binder_physicochemical_signature.png

=== Analysis Complete ===


In [ ]:

# Create a comprehensive summary of the design grammar for competitive binders
print("=== DESIGN GRAMMAR SUMMARY ===\n")

print("=" * 80)
print("HIGH-SCORING COMPETITIVE BINDER DESIGN GRAMMAR")
print("=" * 80)

print("\n1. POSITIVE INDICATORS (features enriched in high-scoring motifs):")
print("   - Net positive charge at pH 7: mean = +0.81 (vs -0.48 in low-scoring)")
print("   - High isoelectric point: mean = 9.03 (vs 5.71 in low-scoring)")
print("   - High positive charge fraction: mean = 0.174 (17.4% K/R/H residues)")
print("   - Low aromatic content: mean = 0.068 (6.8% W/Y/F residues)")
print("   - Hydrophilic character: mean GRAVY = -1.20 (strongly hydrophilic)")
print("   - Minimal negative charge: mean = 0.026 (2.6% D/E residues)")

print("\n2. NEGATIVE INDICATORS (features to avoid):")
print("   - High aromatic content (mean 0.199 in low-scoring)")
print("   - Balanced or negative net charge")
print("   - High negative charge fraction (mean 0.081 in low-scoring)")
print("   - Moderate hydrophobicity (GRAVY closer to 0)")

print("\n3. EXAMPLE HIGH-SCORING MOTIFS:")
top_examples = high_scoring_with_props.nlargest(5, 'residual_score')['motif'].tolist()
for i, motif in enumerate(top_examples, 1):
    print(f"   {i}. {motif}")

print("\n=" * 80)
print("COMPETITIVE vs ALLOSTERIC DESIGN GRAMMAR COMPARISON")
print("=" * 80)

print("\nKEY DIFFERENCES:")
print("\nProperty                    High Competitive   Allosteric      Δ       Cohen's d")
print("-" * 80)
for i, prop in enumerate(properties):
    high_val = high_comp_means[i]
    allo_val = allosteric_means[i]
    diff = high_val - allo_val
    cohen = comp_vs_allo_df[comp_vs_allo_df['Property'] == prop]['Cohens_d'].values[0]
    
    # Format values appropriately
    if prop == 'isoelectric_point':
        print(f"{property_labels[prop]:28s}{high_val:8.2f}          {allo_val:8.2f}     {diff:+7.2f}     {cohen:+6.2f}")
    else:
        print(f"{property_labels[prop]:28s}{high_val:8.3f}          {allo_val:8.3f}     {diff:+7.3f}     {cohen:+6.2f}")

print("\n" + "=" * 80)
print("ACTIONABLE DESIGN PRINCIPLES FOR COMPETITIVE BINDERS")
print("=" * 80)

print("\n1. CHARGE DISTRIBUTION:")
print("   - Target net charge: +0.5 to +1.5 at pH 7")
print("   - Positive charge fraction: 15-20% (enriched in K, R)")
print("   - Negative charge fraction: <5% (minimize D, E)")
print("   - Target pI: 9-11 (basic peptides)")

print("\n2. HYDROPHOBICITY:")
print("   - Target GRAVY score: -1.0 to -1.5 (strongly hydrophilic)")
print("   - Prefer polar residues (S, T, N, Q, Y) over hydrophobic (I, L, V, M)")

print("\n3. AROMATIC CONTENT:")
print("   - Keep aromatic fraction LOW: 0-10% (minimize W, Y, F)")
print("   - This contrasts sharply with allosteric binders (~16.5%)")

print("\n4. SEQUENCE COMPOSITION GUIDELINES:")
print("   - Enriched residues: R, K, S, G, T, Q")
print("   - Depleted residues: W, Y, F, I, L, V, D, E")
print("   - Optimal length range: 8-12 residues (based on data distribution)")

print("\n5. CONTRAST WITH ALLOSTERIC BINDERS:")
print("   - Competitive: Basic, hydrophilic, low-aromatic")
print("   - Allosteric: More neutral charge, higher aromatic content")
print("   - Effect sizes: Large (|d| > 0.8) for all properties except positive charge")

print("\n" + "=" * 80)


=== DESIGN GRAMMAR SUMMARY ===

HIGH-SCORING COMPETITIVE BINDER DESIGN GRAMMAR

1. POSITIVE INDICATORS (features enriched in high-scoring motifs):
   - Net positive charge at pH 7: mean = +0.81 (vs -0.48 in low-scoring)
   - High isoelectric point: mean = 9.03 (vs 5.71 in low-scoring)
   - High positive charge fraction: mean = 0.174 (17.4% K/R/H residues)
   - Low aromatic content: mean = 0.068 (6.8% W/Y/F residues)
   - Hydrophilic character: mean GRAVY = -1.20 (strongly hydrophilic)
   - Minimal negative charge: mean = 0.026 (2.6% D/E residues)

2. NEGATIVE INDICATORS (features to avoid):
   - High aromatic content (mean 0.199 in low-scoring)
   - Balanced or negative net charge
   - High negative charge fraction (mean 0.081 in low-scoring)
   - Moderate hydrophobicity (GRAVY closer to 0)

3. EXAMPLE HIGH-SCORING MOTIFS:
   1. SGSYPTPHN
   2. GSYPTPHN
   3. SGSYPTPH
   4. YSGSYPTPH
   5. RDYQYYYSG

=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
